<a href="https://colab.research.google.com/github/soohyunme/exercise/blob/main/Code/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/4조/01.체육공모전/Code

/content/drive/.shortcut-targets-by-id/159Rw9eptrdiP8wihxnfPLcnRcwjx1x8n/4조/01.체육공모전/Code


# 모듈 로드

In [2]:
import random
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# 데이터 로드

In [3]:
df_origin = pd.read_csv('../Data/04.processed_data.csv',encoding='cp949')
# group = pd.read_csv('../Data/user_group.csv',encoding='cp949')

테스트를 위해 원본 데이터에서 20개만 테스트용으로 분리 후 운동처방정보 삭제

In [4]:
user_db = df_origin[:-20]
sample_user = df_origin[-20:].iloc[:,:-3]

In [5]:
# test input data 저장
sample_user.to_csv('../Data/input.csv',encoding='cp949',index=False)

# 최빈 운동 세트 선정

In [6]:
# exercise set count df
set_df = pd.DataFrame(df_origin[['GROUP_ID','준비운동','본운동','마무리운동']].value_counts()).reset_index().rename(columns={0:'COUNT'})

# 운동 추천 함수

In [7]:
# 운동 리스트를 통해 희소행렬 도출
# input : exercise list / output : sparse matrix
def sparse_matrix(ex_list):
  te = TransactionEncoder()
  te_ary = te.fit(ex_list).transform(ex_list)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_)
  return sparse_matrix

In [8]:
# 연관분석을 통해 선택된 운동과 연관성이 높은 운동 3개를 리턴
def ranking(pre_df, main_df, finish_df, ex_list):
  global df_list
  df_list = [pre_df, main_df, finish_df] # 단계별 연관분석 결과값
  result_list = [] # 결과값을 리스트에 담아 하나씩 출력
  result_df = pd.DataFrame(columns=pre_df.columns) # 빈 DataFrame 생성

  for i in range(len(df_list)): # 운동 단계별 for문 pre, main, finish
    global tmp_df 
    tmp_df = df_list[i] 
    for each_ex in ex_list[i].split(','): # 각 단계별 운동들을 리스트로 분할
      each_ex = each_ex # 운동이름에 공백 제거
      if tmp_df.empty:
        result_df = pd.DataFrame(columns=pre_df.columns)
      else:
        each_result = tmp_df[tmp_df['antecedents']==frozenset({each_ex})].sort_values('support') # 추천 세트 운동과의 연관분석 결과를 지지도 순으로 정렬
        each_result['consequents'] = each_result['consequents'].apply(lambda x : list(x)[0]) # frozenset to string, 타입변환
        length = 3 if len(each_result) else len(each_result) # set length
        result_df = pd.concat([result_df,each_result.iloc[:length]]) # 연관성이 높은 운동이 3개 이상일 경우 3개만 저장
    result_list.append(result_df['consequents']) # 결과값 리스트에 추가
    result_df = None # 초기화
  return list(result_list[0]), list(result_list[1]), list(result_list[2]) # 준비, 본, 마무리 운동 연관성 높은 운동 리턴

In [9]:
# select main routine and additional routine
def select_exercise(pre_list, main_list, finish_list, max):
  main_reco_list = [] # 메인 추천 리스트
  sub_reco_list = [] # 추가 추천 리스트
  tmp_list = []
  asso_list = [pre_list,main_list,finish_list] # 연관 있는 운동 리스트
  tmp_set = set()
  random_num = random.randint(1, 100) # 임의의 1개 운동을 선택하기 위한 난수 생성
  
  for i, str in enumerate(max):
    ex_list = str.split(',') # 운동 세트에서 각 단계별 운동들
    reco_ex = ex_list[random_num %len(ex_list)] # 난수를 통해 임의의 1개 운동 선택
    main_reco_list.append(reco_ex) # 메인 추천 리스트에 선택된 운동 삽입

    tmp_set.update(ex_list) # 임시 리스트에 max 운동 리스트 삽입
    tmp_set.remove(reco_ex) # 선택된 운동 삭제 -> 단계별 최빈 세트 운동에서 메인 추천 운동 제거
    tmp_set.update(asso_list[i]) # 연관 분석을 통해 추천된 운동들 추가
    tmp_list.append(list(tmp_set)) # 해당 set을 리스트로 변환하여 tmp_list에 추가 -> [ [준,], [본,], [마,] ]
  for i, reco in enumerate(tmp_list):
    if (len(reco)) == 0:
      random_list = ['없음']
    elif (len(reco)) < 4:
      random_list = reco # 운동 개수가 4개 이하면 전체 운동 적용
    else:
      random_list = random.sample(reco, 4) # 운동 개수가 4개 초과일 경우 임의로 4개 선택
    sub_reco_list.append(random_list)
  return list(main_reco_list), list(sub_reco_list)

In [10]:
# select top 1 exercise set from user info
# input : group_id / ouput : recommend_set, sub_recommend_set
def mode_set(g_id, min_support = 0.01, min_threshold=0.2):
  user_set = set_df[set_df['GROUP_ID']==g_id] # 유저가 해당하는 그룹의 처방 운동 세트 빈도
  max_ex_set = list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]) # 가장 많이 처방된 운동 세트

  # 해당 그룹의 단계별 운동 리스트 생성
  pre_ex_all = list(user_set['준비운동'].apply(lambda x : x.split(','))) 
  main_ex_all = list(user_set['본운동'].apply(lambda x : x.split(',')))
  finish_ex_all = list(user_set['마무리운동'].apply(lambda x : x.split(',')))

  # 단계별 sparse matrix 생성
  pre_sparse = sparse_matrix(pre_ex_all)
  main_sparse = sparse_matrix(main_ex_all)
  finish_sparse = sparse_matrix(finish_ex_all)

  # 지지도가 0.01 이상인 운동 선택
  pre_itemsets = apriori(pre_sparse, min_support = min_support, use_colnames=True,max_len=2)
  main_itemsets = apriori(main_sparse, min_support = min_support, use_colnames=True,max_len=2)
  finish_itemsets = apriori(finish_sparse, min_support = min_support, use_colnames=True,max_len=2)

  # 그 중에서 신뢰도가 0.2 이상인 운동 선택
  pre_select = association_rules(pre_itemsets, metric="confidence", min_threshold=min_threshold)
  main_select = association_rules(main_itemsets, metric="confidence", min_threshold=min_threshold)
  finish_select = association_rules(finish_itemsets, metric="confidence", min_threshold=min_threshold)

  # consequents의 길이가 1인 것만 선택 -> 1개의 운동만 추천하도록
  pre_select = pre_select[pre_select['consequents'].apply(lambda x : len(x) == 1)]
  main_select = main_select[main_select['consequents'].apply(lambda x : len(x) == 1)] 
  finish_select = finish_select[finish_select['consequents'].apply(lambda x : len(x) == 1)]
  
  # ranking 
  pre_rank, main_rank, finish_rank = ranking(pre_select, main_select, finish_select, max_ex_set)

  # select exercise 
  recommend_set, sub_recommend_set = select_exercise(pre_rank, main_rank, finish_rank, max_ex_set)

  return recommend_set, sub_recommend_set


# 맞춤 운동 추천

In [11]:
# 변수 선언 및 결과 df 생성
main_reco = []
sub_reco = []
u_id_list = []
result_df = sample_user.copy()

In [12]:
for u_id, g_id in sample_user[['USER_ID','GROUP_ID']].values:
  best, sub = mode_set(g_id) # 추천 루틴 및 추가 추천 운동
  main_reco.append(list(best)) 
  sub_reco.append(sub)
  u_id_list.append(u_id)

In [13]:
# DataFrame 생성
reco_df = pd.DataFrame(main_reco,index=u_id_list,columns=['추천준비운동','추천본운동','추천마무리운동'])
sub_df = pd.DataFrame(sub_reco,index=u_id_list,columns=['추가준비운동','추가본운동','추가마무리운동'])
merge_df = pd.merge(reco_df,sub_df,left_index=True,right_index=True).reset_index().rename(columns={'index':'USER_ID'})
result_df = pd.merge(sample_user,merge_df)


In [15]:
# 결과 저장
result_df.to_csv('../Data/result.csv',encoding='cp949',index=False)


# 추천 받아보기

In [ ]:
age = int(input('나이 : '))
cert = input('운동강도(상/중/하) :')
sex = input('성별(F/M) :')
height = np.float64(round(float(input('키 :')),1))
weight = np.float64(round(float(input('몸무게 :')),1))

나이 : 27
운동강도(상/중/하) :하
성별(F/M) :M
키 :180
몸무게 :100


In [ ]:
input_list = [22222,age,'하',sex,height,weight]
input_df = pd.DataFrame(input_list,index=sample_user.columns).T.astype(sample_user.dtypes)
trans_input = transform(input_df)
mode_set(int(trans_input.GROUP_ID))

(['걷기', '무릎 높여 제자리 달리기', '전신 루틴 스트레칭'],
 [['자전거타기'],
  ['윗몸올리기', '계단 올라갔다 내려오기', '달리기', '엎드려 버티기'],
  ['하지 루틴 스트레칭', '사다리 안팎 뛰기', '자전거타기', '엎드려 버티기']])